In [27]:
# Import packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from patsy import dmatrices

In [28]:
dta = pd.read_csv('../data/weekData.csv')

In [29]:
dta.describe()

,Unnamed: 0,household_key,QUANTITY,BASE_SPEND_AMT,NET_SPEND_AMT,LOY_CARD_DISC,COUPON_DISC,GET_EGGS,WEEK,NEXT_EGGS,X,BASKET_ID,PRODUCT_ID
count,27076.000000,27076.000000,27076.000000,27076.000000,27076.000000,27076.000000,27076.000000,27076.000000,27076.000000,27076.000000,27076.000000,2.707600e+04,2.707600e+04
mean,13538.500000,504.015512,13.229908,38.945442,33.333592,-5.479414,-0.132436,0.217868,14.500000,0.133033,27661.825233,3.568014e+10,1.196953e+06
std,7816.312281,289.027343,21.843034,65.189523,55.231095,10.510180,0.971411,0.412805,8.077896,0.374280,45630.758174,1.990048e+09,1.682116e+06
min,1.000000,1.000000,0.000000,0.000000,0.000000,-455.260000,-48.880000,0.000000,1.000000,0.000000,1.000000,3.457753e+10,3.291600e+04
25%,6769.750000,253.000000,0.000000,0.000000,0.000000,-6.970000,0.000000,0.000000,7.750000,0.000000,3019.000000,3.464284e+10,8.313900e+05
50%,13538.500000,506.000000,2.000000,5.080000,4.575000,0.000000,0.000000,0.000000,14.500000,0.000000,9927.000000,3.483741e+10,8.567160e+05
75%,20307.250000,756.000000,19.000000,54.510000,46.905000,0.000000,0.000000,0.000000,21.250000,0.000000,28176.000000,3.538682e+10,9.294330e+05
max,27076.000000,1000.000000,286.000000,1551.110000,1046.970000,0.000000,0.000000,1.000000,28.000000,4.000000,278461.000000,4.209784e+10,1.574167e+07


In [30]:
dta = dta.drop('Unnamed: 0', 1)

In [31]:
# Create X and y matrices
# X is the input feature matrix
# y is the column vector of the parameter we want to predict

y, X = dmatrices('NEXT_EGGS ~ QUANTITY + BASE_SPEND_AMT + NET_SPEND_AMT + LOY_CARD_DISC + COUPON_DISC + GET_EGGS', dta, return_type="dataframe")

In [32]:
# Percentage of data with eggs

print(y['NEXT_EGGS'].value_counts())
freq = y.sum(axis=0)[0] / y.shape[0]
print(freq)

0.0    23783
1.0     3007
2.0      266
3.0       17
4.0        3
Name: NEXT_EGGS, dtype: int64
0.133032944305


In [33]:
# Cross Validation folds with 20/80 split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [34]:
# Baseline predictions 
# Guessing 0 for everything

baselinepredict = [0 for x in range(0, y_test.shape[0])]

In [35]:
# Briar score on the xvaldiation

metrics.brier_score_loss(y_test, baselinepredict)

0.0009231905465288035

In [36]:
# Logistic regression model

model = LogisticRegression()
model.fit(X_train, y_train)
logisticpredict = model.predict(X_test)

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [37]:
# How many 1's and 0's of the prediction

print(np.count_nonzero(logisticpredict==1))
print(np.count_nonzero(logisticpredict==0))

800
4616


In [38]:
# Briar score on the xvalidation

metrics.brier_score_loss(y_test, logisticpredict)

0.1475258493353028

In [39]:
# Random forest model

model2 = RandomForestClassifier()
model2.fit(X_train, y_train)
forestpredict = model2.predict(X_test)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [40]:
# How many 1's and 0's of the prediction

print(np.count_nonzero(forestpredict==1))
print(np.count_nonzero(forestpredict==0))

570
4839


In [41]:
# Briar score on the xvalidation

metrics.brier_score_loss(y_test, forestpredict)

ValueError: y_prob contains values greater than 1.

In [42]:
import xgboost as xgb

In [43]:
# XGB Boost model

gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(X_train, y_train)
xgbpredict = gbm.predict(X_test)

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:108: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [44]:
# How many 1's and 0's of the prediction

print(np.count_nonzero(xgbpredict==1))
print(np.count_nonzero(xgbpredict==0))

505
4911


In [45]:
metrics.brier_score_loss(y_test, xgbpredict)

0.093057607090103397

In [47]:
df = pd.read_csv('../data/finalWeek.csv')


In [56]:
yans, Xans = dmatrices('NEXT_EGGS ~ QUANTITY + BASE_SPEND_AMT + NET_SPEND_AMT + LOY_CARD_DISC + COUPON_DISC + GET_EGGS', df, return_type="dataframe")

In [57]:
anspredict = gbm.predict(Xans)

In [58]:
print(len(Xans))

967


In [53]:
print(np.count_nonzero(anspredict==1))
print(np.count_nonzero(anspredict==0))

2386
24689


In [26]:
np.savetxt("../predictions.csv", xgbpredict, delimiter=",")